# MO cheating


## MO

In [17]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
from MO_agent import *
import random
import agent as agent

===============================================================================

In [1]:


e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)
e.reset()
initial_observation=[e.current_state]
observation=initial_observation
state=e.current_state
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 1, 'initial_wealth': 0}


In [2]:
# parameters
exp_vorfaktoren=[0.2,0.3]
exp_weights=[2,1]
partitioning_chunk_number=1000
initial_theta=[0.5,0.5]
initial_observation=0
planning_depth=2
maximum_depth=planning_depth

internal_state=initial_theta*len(exp_vorfaktoren)
internal_state.extend([initial_observation])

In [3]:
ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='optimized')
ag.reset()
_,M,F,G,X,value_function,all_theta=ag.pre_planning(exp_vorfaktoren, initial_theta=initial_theta,initial_observation=initial_observation)

### Size of reachable states

In [4]:
x0=ag.initialize_internal_state(theta_0=initial_theta, observation_0=initial_observation, exp_num=len(exp_vorfaktoren))
continous_rachable_states=[]
intial_x_state=x0
env_dynamics=e
max_depth=planning_depth
exp_num=len(exp_vorfaktoren)




reachables={}
reachables[0]=[tuple(intial_x_state)]

num_actions=len(env_dynamics.actions)
num_observations=len(env_dynamics.observations)
observations=list(env_dynamics.observations.keys())

universal_map={}

# until the last step
for depth in range(1,max_depth+1):
    reachables[depth]=[]
    for to_extend in reachables[depth-1]:      
        to_extend=np.array(to_extend)
        this_extention=[]
        for a in range(num_actions):  

            for y_prim in range(num_observations):
                next_x=[]
                for i in range(exp_num):

                    
                    z=np.matmul(M[i,a,y_prim,:,:],to_extend[i*2:(i+1)*2].reshape(2,1))
                    integral=np.sum(z)
                    
                    
                    next_theta=z/integral
                    

                    next_x.extend(next_theta)


                next_x.append(y_prim)

                new_entry=np.array(next_x)
                
                is_unique=True
                for rec in reachables[depth]:
                    rec=np.array(rec)
                    if (np.max(np.abs(rec-new_entry))<0.001):
                        is_unique=False
                if is_unique:
                    new_entry*=100000
                    new_entry=new_entry.astype(np.int64)
                    new_entry=new_entry.astype(np.float64)/100000
                    reachables[depth].append(tuple(new_entry))
                
                universal_map[(tuple(to_extend),a,y_prim)]= tuple(new_entry)

             
                    



C:\Users\arsha\AppData\Local\Temp\ipykernel_12516\942802216.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_entry=np.array(next_x)


In [5]:
for i in range(len(reachables)):
    print('i:',len(reachables[i]))

i: 1
i: 4
i: 8


### Value iteration

In [6]:
discount=env_dynamics.discount_factor
reachable_states=reachables

# backwardly
q_func={}
action_func=[0]*(max_depth)
value_function=[0]*(max_depth)

#for step in range(max_depth-1,-1,-1):
for step in range(max_depth-1,-1,-1):
    this_step=list(map(list,reachable_states[step])).copy()

    q_tmp=np.empty((len(this_step),num_actions))
    for a in range(num_actions):
        this_action_q=np.zeros((len(this_step),1))

        for y_prim in range(num_observations):
            this_y_prim_nextState=np.empty((len(this_step),2))
            this_y_prim_cost=np.zeros((len(this_step),1))
            for i in range(exp_num):

                this_step=np.array(this_step)

                theta_i=this_step[:,i*2:2*i+2].copy().T

                z=np.matmul(M[i,a,y_prim,:,:],theta_i)
                z=z.T
                
                integral=np.sum(z,axis=1).reshape(len(z),1)          
                # F- function
                next_theta_i=(z/integral)
                
                
                
                # next state
                next_theta_i*=100000
                next_theta_i=next_theta_i.astype(np.int64)
                next_theta_i=next_theta_i.astype(np.float64)/100000
                
                # cost function
                g_i=np.log(integral)/exp_vorfaktoren[i]
                c_i=g_i +np.log(np.power(num_observations,1/exp_vorfaktoren[i])) 
                #c_i=g_i +np.power(np.log(num_observations),1/exp_vorfaktoren[i])
                #c_i=g_i +np.log(num_observations)       
                
                # utility of cost
                this_y_prim_cost=this_y_prim_cost+exp_weights[i]*np.exp(exp_vorfaktoren[i]*c_i)
                #print('step:',step,'a:',a, 'y_prime:',y_prim, 'i:',i, 'C_i:',c_i,exp_weights[i]*np.exp(exp_vorfaktoren[i]*c_i))
                

                if i==0:

                    this_y_prim_nextState=next_theta_i
                else:
                    this_y_prim_nextState=np.hstack((this_y_prim_nextState,next_theta_i))
                    
                    
            this_y_prim_nextState=np.concatenate((this_y_prim_nextState,np.array([y_prim]*len(this_step)).reshape(len(this_step),1)),axis=1)

            # if max step
            if step==max_depth-1:

                this_action_q+=this_y_prim_cost * (1./num_observations)
                #if y_prim==1:
                    #print(a,this_action_q)
                #print(a,y_prim,this_y_prim_cost)
            else:
                # else
                next_v=[]
                for r,thet in enumerate(this_y_prim_nextState):
                    thet=tuple(thet)

                    next_v.append(value_function[step+1][thet])  
                next_v=np.array(next_v).reshape(len(this_step),1)
                this_action_q+= (this_y_prim_cost + discount* next_v) * (1./num_observations)   # * (1./num_observations)


        q_tmp[:,a]= this_action_q.reshape(len(this_step),)
        # choose best action, record values, record actions
        # check to see if it has equal values for action=1,2 and 3,4 again

    best_actions=q_tmp.argmax(axis=1)
    best_values=q_tmp.max(axis=1)
    
    q_func[step]={}
    action_func[step]={}
    value_function[step]={}
    #value_function.append({})
    
    for p,point in enumerate(this_step):
        point=tuple(point)
        
        q_func[step][point]=q_tmp[p].copy()
        action_func[step][point]=best_actions[p].copy()
        value_function[step][point]=best_values[p].copy()

    # update the values

    #value_function[step]= dict((tuple(this_step[k]),best_values[k]) for k in range(len(best_values)))



In [7]:
action_func

[{(0.5, 0.5, 0.5, 0.5, 0.0): 3},
 {(0.7, 0.3, 0.7, 0.29999, 0.0): 4,
  (0.3, 0.7, 0.29999, 0.7, 1.0): 3,
  (0.5, 0.5, 0.5, 0.5, 0.0): 3,
  (0.5, 0.5, 0.5, 0.5, 1.0): 3}]

In [21]:
action_func

{1: array([{(0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.30000000000000004, 0.0, 0.0): 4, (0.0, 0.0, 0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0): 3, (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0): 3, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25): 3},
        {(0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.30000000000000004, 0.0, 0.0): 4, (0.0, 0.0, 0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0): 3, (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0): 3, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25): 3}],
       dtype=object),
 0: array([{(0.5, 0.5): 3}, {(0.5, 0.5): 3}], dtype=object)}

In [12]:
ff

{2: [{(0.936619718309859, 0.06338028169014084): 1.889451728165305,
   (0.8663366336633662, 0.13366336633663364): 1.8894517281653047,
   (0.7, 0.30000000000000004): 1.8894517281653052,
   (0.7307692307692308, 0.2692307692307692): 1.8894517281653052,
   (0.5434782608695652, 0.45652173913043465): 1.8894517281653047,
   (0.30000000000000004, 0.7): 1.8894517281653052,
   (0.5, 0.5): 1.8894517281653052}],
 1: [{(0.9545454545454545,
    0.04545454545454545): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173]),
   (0.7941176470588235,
    0.20588235294117643): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173]),
   (0.5,
    0.5): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173])}],
 0: [{(1,
    0): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173])}]}

In [11]:
b_reachables

{0: [(1, 0)],
 1: [(1.0, 0.0), (0.5, 0.5)],
 2: [(1.0, 0.0), (0.7, 0.3), (0.3, 0.7), (0.5, 0.5)]}

In [13]:
Ss

{0: [0], 1: [-1.0], 2: [-2.0]}

In [14]:
b_utility(x=[1,0],exp_weights=[2,1],exp_vorfaktoren=[0.2,0.3])

array([[3.79266432, 3.        ]])

# Beurele Cheatnig

In [8]:

maximum_depth=2
num_of_Mu_chunks=3
exp_vorfaktoren=[0.2,0.3]
exp_weights=[2,1]


e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)
e.reset()


{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 1, 'initial_wealth': 0}


In [11]:
# reachables Mu-states in each time step. Here, like the complete version, observations (observable states in the paper) are not important for the transitions therefore we exclude them form the extended states. Also,the discount factor is coeded in the time step.
b_reachables={}
# Transition Kernel. a mapping from (Mu-states, action,next_observation) to the (next Mu-state). 
TK={}

initial_wealth=0
env=e
max_depth=maximum_depth

initial_mu_state=[0.5,0.5]



num_actions=len(env.actions)
num_observations=len(env.observations)
observations=list(env.observations.keys())

In [12]:
ag=agent.Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=num_of_Mu_chunks,max_iterations=maximum_depth)
Ss=ag.generate_possible_wealths(np.unique(env.rewards),initial_wealth,env.discount_factor,max_depth,comparison_precision=1.0e-5)

### Reachable Mu space points

In [13]:
# q(x_prim, y_prim | x,y,a) while in our settign it is equal to q(x_prim,y_prim|y,a)
# make Q-kernel, the probability of reaching each y_prim x_prim pair when the real state is y and doing action a
q=ag.make_Q_kernel(env.transition_matrix,env.observation_matrix)

# q(x_prim | x,y,a) while in our settign it is equal to q(x_prim|y,a)
# make marginal-Q-kernel, the probability of getting observation x_prim, when the real state is y and doing action a
mq=ag.make_marginal_Q_kernel(env.transition_matrix,env.observation_matrix)




#for step in range(maximum_depth+1):
for step in range(maximum_depth):
    
    if step==0:
        b_reachables[0]=[tuple(initial_mu_state)]
    
    thisStep_s=Ss[step]
    ls=len(thisStep_s)
    mus=np.reshape(b_reachables[step],(len(b_reachables[step]),len(b_reachables[step][0])))
    #mus=mus[:,1]
    #mus=np.array(list(map(list,mus)))
    z=env.discount_factor**step
    
    all_current_s=thisStep_s
    all_next_s=Ss[step+1]


    # decompose Mu-beliefs about being in each state (y=0 or 1)
    mus_y0=mus[:,0:ls]
    mus_y1=mus[:,ls:2*ls]
    mus_ys=[mus_y0,mus_y1]

    # Marginal Mu (Mu superscript Y in the paper, or Mu(dy,R)). This variable expresses the probability of being in each state(y)
    marginal_mus_y0=mus_y0.sum(axis=1)
    marginal_mus_y1=mus_y1.sum(axis=1)
    marginal_mus_ys=[marginal_mus_y0,marginal_mus_y1]
  
    
    nextStep_uniques=[]
    b_reachables[step+1]=[]
    TK[step]={}
        
        
    #for action in range(num_actions):
    for action in range(num_actions):
        for x_prim in range(num_observations):
            ########################################################
            
            ## prepare
            
            # C(x,y,a) while C depends only on y
            #rewards/costs of doing action a when the real state is y 
            y0=0
            y1=1
            y_prim0=0
            y_prim1=1
            c_y0=env.rewards[y0][action]
            c_y1=env.rewards[y1][action]
            c=[c_y0,c_y1]
            
            ### Calculate the denominator of SAY function
        
            # It is equal to probaility of recieving observation x_prim, regardless of what are the wealth(s) or the state(y). 
            # It calculate sum of probabilities of reaching each state (regardless of wealth level) (marginal_mus_y), while reaciving x_prime observation
            say_denominator=marginal_mus_y0*mq[0][action][x_prim]+marginal_mus_y1*mq[1][action][x_prim]
            
            
            ### calculate the nomerator of the SAY function
        
            # The dimensions of the Mu-space doesn't change with just one action and one observation.  Because in our experiment the reward function is deterministic, all of possible wealths of this step, will transfer to just one other value 
            # based on state and action. So, the size of Mu-space remains constant in SAY calculator function (for doing only one action). Also, the number of possible distributions over Mu, is not a concern for this function: It maps all current possible values to
            # continious values

            ## allocate variables for results of nomerator calculations for each current state

            # these arrays are here to represent Mu distribution of the next state (naturally over its own (next state's) wealth levels) 
            for_y0=np.zeros((len(mus),len(all_next_s)*2))
            for_y1=np.zeros((len(mus),len(all_next_s)*2))
            next_mus=[for_y0,for_y1]

            # tmp_mus[0] for y=0 and tmp_mus[1] for y=1 calculations
            tmp_mus=np.zeros((2,len(mus),len(mus[0])))
            
            ##
             # for each current state
            for y,y_mus in enumerate(tmp_mus):
                
                # mus_ys is Mu-beliefs about being in a staet (y)
                # probability of reaching y_prim=0
                y_mus[:,:len(all_current_s)]=mus_ys[y]*q[y][action][y_prim0][x_prim]
                # probability of reaching y_prim=1
                y_mus[:,len(all_current_s):]=mus_ys[y]*q[y][action][y_prim1][x_prim]

                # Dirac function: d(s+zc(x,y,a))
                # Here, we compute the possible values of s for next Mu distribution         
                next_possible_s=np.array(all_current_s)+z*c[y]

                #The calculated Mu distributions on the S-axis, are defined on the current S values, however these probabilities are for s+zc(y,a).
                # so as the SAY function inputs are (x,a,x_prim), for each y we can rotate Mu(s) to be matched with next stage's possible values

                ### indexes of S axis which has value in this SAY calculation
                ##current_s_indexes=np.where(np.isin(self.S,current_possible_s)==True)[0]

                # indexes of the next wealth levels which are the successors ( after current S-values recieving c(y,a) )
                # Here, we used comparison method to check the equality of next time-spte's S-values and current S-values+ z*c 
                next_s_indexes=np.empty(0,np.int32)
                for ind,ns in enumerate(next_possible_s):
                    next_related_index=np.where(np.abs(np.array(all_next_s)-ns)<1.0e-5)[0][0]
                    next_s_indexes=np.append(next_s_indexes,next_related_index)


                #assign Mu-probability to each possible S(=previous_s + c) point
                # for S points in y_prim=0        
                next_mus[y][:,next_s_indexes]=y_mus[:,:len(all_current_s)]
                # for S points in y_prim=1
                next_mus[y][:,next_s_indexes+len(all_next_s)]=y_mus[:,len(all_current_s):]


            # sum of probabilities of next Mu-space for both conditions : p(mu|y0), p(mu|y1)
            say_nomerator=next_mus[0]+next_mus[1]

            # calculate final SAY result
            # normalize the whole Mu-space dist. by dividing it by totoal probability of taking x_prim
            say_result=say_nomerator/say_denominator[:,None]  
            
            ### make extended states and transition kernel between them
            for p in range(len(say_result)):
 
                
                ## reachable extended states
                redundants=[]
                is_unique=True
                for i in range(len(nextStep_uniques)):
                    if np.sum(np.abs(np.array(say_result[p])-np.array(nextStep_uniques[i])))<1.0e-5:
                        is_unique=False
                        redundants.append(i)
                        
                if is_unique:
                    
                    # add unique values for other comparisons
                    #nextStep_uniques[0].extend([x_prim])
                    nextStep_uniques.append(say_result[p])
                    
                    # adding to extended reachable states
                    b_reachables[step+1].append((tuple(say_result[p])))
                    
                    ######### transition kernel of extended states
                    
                    ex_state=tuple(say_result[p])
                    # here x_prime means x. (should be checked)
                    ex_kernel_element=(x_prim,b_reachables[step][p],action)

                    TK[step][ex_kernel_element]=ex_state
                else:
                    ## transition kernel of extended states
                    
                    ex_state=tuple(nextStep_uniques[redundants[0]])
                    # here x_prime means x. (should be checked)
                    ex_kernel_element=(x_prim,b_reachables[step][p],action)

                    TK[step][ex_kernel_element]=ex_state
    

In [14]:
for i in range(maximum_depth+1):
    print('i:',len(b_reachables[i]))

i: 1
i: 4
i: 15


In [15]:
for i in (TK[0]):
    print(i,'===',TK[0][i])

(0, (0.5, 0.5), 0) === (0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.30000000000000004, 0.0, 0.0)
(1, (0.5, 0.5), 0) === (0.0, 0.0, 0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0)
(0, (0.5, 0.5), 1) === (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0)
(1, (0.5, 0.5), 1) === (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0)
(0, (0.5, 0.5), 2) === (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0)
(1, (0.5, 0.5), 2) === (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0)
(0, (0.5, 0.5), 3) === (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25)
(1, (0.5, 0.5), 3) === (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25)
(0, (0.5, 0.5), 4) === (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25)
(1, (0.5, 0.5), 4) === (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25)


## Value Iteration

In [18]:
value_func={}
action_func={}
full_val_func={}

for step in range(maximum_depth,-1,-1):
    points=b_reachables[step]
    #value_func[step]={}
    #full_val_func[step]={}
    if step==maximum_depth:
        each_wealth_level_value=b_utility(Ss[step],exp_weights,exp_vorfaktoren)
        
        each_mu_value=np.dot(np.array(list(map(list,points))),np.hstack([each_wealth_level_value,each_wealth_level_value]).T)
        
        value_func[step]=np.hstack([dict(zip(points,each_mu_value.reshape(-1))).copy(),dict(zip(points,each_mu_value.reshape(-1))).copy()])
        
        full_val_func[step]=[dict(zip(points,each_mu_value.reshape(-1))).copy()]
    else:
        val_x0=np.zeros((len(points),num_actions))
        val_x1=np.zeros((len(points),num_actions))
        val_x=[val_x0,val_x1]
        for x in observations:
            for a in range(num_actions):
                for p,point in enumerate(points):
                    
                    val_x[x][p][a]=value_func[step+1][x][TK[step][(x,point,a)]]
            
            to_v0=np.max(val_x0,axis=1)
            to_a0=np.argmax(val_x0,axis=1)
            
            to_v1=np.max(val_x1,axis=1)
            to_a1=np.argmax(val_x1,axis=1)
        
        value_func[step]=np.hstack([dict(zip(points,to_v0)).copy(),dict(zip(points,to_v1)).copy()])
        action_func[step]=np.hstack([dict(zip(points,to_a0)).copy(),dict(zip(points,to_a1)).copy()])
        full_val_func[step]=[dict(zip(points,val_x0)).copy()]
        
    
        

In [9]:
full_val_func[0]

[{(1, 0): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173])}]

In [10]:
ff=full_val_func.copy()

In [20]:
ff

{2: [{(1.0, 0.0): 1.8894517281653052,
   (0.7, 0.3): 1.8894517281653052,
   (0.3, 0.7): 1.8894517281653052,
   (0.5, 0.5): 1.8894517281653052}],
 1: [{(1.0,
    0.0): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173]),
   (0.5,
    0.5): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173])}],
 0: [{(1,
    0): array([1.88945173, 1.88945173, 1.88945173, 1.88945173, 1.88945173])}]}

============================================================================================================================================================================

# From here:

## imports

In [2]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
from MO_agent import *
import random
import agent as agent

## Functions

## MO

In [3]:
def MO_cheating(env_dynamics,planning_depth,exp_vorfaktoren,exp_weights ,initial_theta=[0.5,0.5],initial_observation=0,rounding_prec_coeff=10000):   
    
    partitioning_chunk_number=1000
    internal_state=initial_theta*len(exp_vorfaktoren)
    internal_state.extend([initial_observation])
    ag=Multi_Variate_agent(environment=env_dynamics, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='optimized')
    ag.reset()
    _,M,F,G,X,value_function,all_theta=ag.pre_planning(exp_vorfaktoren, initial_theta=initial_theta,initial_observation=initial_observation)
    
    intial_x_state=ag.initialize_internal_state(theta_0=initial_theta, observation_0=initial_observation, exp_num=len(exp_vorfaktoren))
    
    max_depth=planning_depth
    exp_num=len(exp_vorfaktoren)


    ######################################################################################## complete forward search

    reachables={}
    reachables[0]=[tuple(intial_x_state)]

    num_actions=len(env_dynamics.actions)
    num_observations=len(env_dynamics.observations)
    observations=list(env_dynamics.observations.keys())

    universal_map={}

    # until the last step
    for depth in range(1,max_depth+1):
        reachables[depth]=[]
        for to_extend in reachables[depth-1]:      
            to_extend=np.array(to_extend)
            this_extention=[]
            for a in range(num_actions):  

                for y_prim in range(num_observations):
                    next_x=[]
                    for i in range(exp_num):


                        z=np.matmul(M[i,a,y_prim,:,:],to_extend[i*2:(i+1)*2].reshape(2,1))
                        integral=np.sum(z)


                        next_theta=z/integral
                        
                        next_theta=next_theta.reshape(-1)
                        next_theta=next_theta.tolist()
                        next_x.extend(next_theta)
                        


                    next_x.append(y_prim)
                    
                    new_entry=np.array(next_x)
                    
                    is_unique=True
                    for rec in reachables[depth]:
                        rec=np.array(rec)
                        if (np.max(np.abs(rec-new_entry))<1./(10*rounding_prec_coeff)):
                            is_unique=False
                    if is_unique:
                        new_entry*=rounding_prec_coeff
                        new_entry=new_entry.astype(np.int64)
                        new_entry=new_entry.astype(np.float64)/rounding_prec_coeff
                        reachables[depth].append(tuple(new_entry))

                    universal_map[(tuple(to_extend),a,y_prim)]= tuple(new_entry)
    
    ################################################################################################# VI
    discount=env_dynamics.discount_factor
    reachable_states=reachables

    # backwardly
    q_func={}
    action_func=[0]*(max_depth)
    value_function=[0]*(max_depth)

    #for step in range(max_depth-1,-1,-1):
    for step in range(max_depth-1,-1,-1):
        this_step=list(map(list,reachable_states[step])).copy()

        q_tmp=np.empty((len(this_step),num_actions))
        for a in range(num_actions):
            this_action_q=np.zeros((len(this_step),1))

            for y_prim in range(num_observations):
                this_y_prim_nextState=np.empty((len(this_step),2))
                this_y_prim_cost=np.zeros((len(this_step),1))
                for i in range(exp_num):

                    this_step=np.array(this_step)

                    theta_i=this_step[:,i*2:2*i+2].copy().T

                    z=np.matmul(M[i,a,y_prim,:,:],theta_i)
                    z=z.T

                    integral=np.sum(z,axis=1).reshape(len(z),1)          
                    # F- function
                    next_theta_i=(z/integral)



                    # next state
                    next_theta_i*=rounding_prec_coeff
                    next_theta_i=next_theta_i.astype(np.int64)
                    next_theta_i=next_theta_i.astype(np.float64)/rounding_prec_coeff

                    # cost function
                    g_i=np.log(integral)/exp_vorfaktoren[i]
                    c_i=g_i +np.log(np.power(num_observations,1/exp_vorfaktoren[i])) 
                    #c_i=g_i +np.power(np.log(num_observations),1/exp_vorfaktoren[i])
                    #c_i=g_i +np.log(num_observations)       

                    # utility of cost
                    this_y_prim_cost=this_y_prim_cost+exp_weights[i]*np.exp(exp_vorfaktoren[i]*c_i)
                    #print('step:',step,'a:',a, 'y_prime:',y_prim, 'i:',i, 'C_i:',c_i,exp_weights[i]*np.exp(exp_vorfaktoren[i]*c_i))


                    if i==0:

                        this_y_prim_nextState=next_theta_i
                    else:
                        this_y_prim_nextState=np.hstack((this_y_prim_nextState,next_theta_i))


                this_y_prim_nextState=np.concatenate((this_y_prim_nextState,np.array([y_prim]*len(this_step)).reshape(len(this_step),1)),axis=1)

                # if max step
                if step==max_depth-1:

                    this_action_q+=this_y_prim_cost * (1./num_observations)
                    #if y_prim==1:
                        #print(a,this_action_q)
                    #print(a,y_prim,this_y_prim_cost)
                else:
                    # else
                    next_v=[]
                   
                    for r,thet in enumerate(this_y_prim_nextState):
                        thet=tuple(thet)
                        if (exp_vorfaktoren[0]==-1.5 and exp_vorfaktoren[1]==-1.5 and exp_weights[0]==-1 and exp_weights[1]==-1 and initial_theta[0]==0.75):
                            pass
                        next_v.append(value_function[step+1][thet])  
                    next_v=np.array(next_v).reshape(len(this_step),1)
                    this_action_q+= (this_y_prim_cost + discount* next_v) * (1./num_observations)   # * (1./num_observations)


            q_tmp[:,a]= this_action_q.reshape(len(this_step),)
            # choose best action, record values, record actions
            # check to see if it has equal values for action=1,2 and 3,4 again

        best_actions=q_tmp.argmax(axis=1)
        best_values=q_tmp.max(axis=1)

        q_func[step]={}
        action_func[step]={}
        value_function[step]={}
        #value_function.append({})

        for p,point in enumerate(this_step):
            point=tuple(point)

            q_func[step][point]=q_tmp[p].copy()
            action_func[step][point]=best_actions[p].copy()
            value_function[step][point]=best_values[p].copy()

        # update the values

        #value_function[step]= dict((tuple(this_step[k]),best_values[k]) for k in range(len(best_values)))
    
    return reachables,universal_map, q_func,value_function,action_func


    


## Beurele

In [4]:
def b_utility(x,exp_weights,exp_vorfaktoren):
    terms_x=np.exp(np.matmul(np.array(exp_vorfaktoren).reshape(len(exp_vorfaktoren),1),np.array(x).reshape(1,len(x))))
    return np.dot(terms_x.T,np.array(exp_weights).reshape(len(exp_weights),1)).T

In [5]:
def Beurele_cheating(env_dynamics,planning_depth,exp_vorfaktoren,exp_weights ,initial_mu_state=[0.5,0.5],initial_wealth=0,comparison_precision=1.0e-5):   
    num_of_Mu_chunks=3
    
    # reachables Mu-states in each time step. Here, like the complete version, observations (observable states in the paper) are not important for the transitions therefore we exclude them form the extended states. Also,the discount factor is coeded in the time step.
    b_reachables={}
    # Transition Kernel. a mapping from (Mu-states, action,next_observation) to the (next Mu-state). 
    TK={}

    env=env_dynamics
    maximum_depth=planning_depth

    num_actions=len(env.actions)
    num_observations=len(env.observations)
    observations=list(env.observations.keys())
    
    ag=agent.Bauerle_Rieder_agent(environment=env, num_of_Mu_chunks=num_of_Mu_chunks,max_iterations=maximum_depth)
    Ss=ag.generate_possible_wealths(np.unique(env.rewards),initial_wealth,env.discount_factor,maximum_depth,comparison_precision=comparison_precision)
    
    ############################################################### complete forward search
    
    # q(x_prim, y_prim | x,y,a) while in our settign it is equal to q(x_prim,y_prim|y,a)
    # make Q-kernel, the probability of reaching each y_prim x_prim pair when the real state is y and doing action a
    q=ag.make_Q_kernel(env.transition_matrix,env.observation_matrix)

    # q(x_prim | x,y,a) while in our settign it is equal to q(x_prim|y,a)
    # make marginal-Q-kernel, the probability of getting observation x_prim, when the real state is y and doing action a
    mq=ag.make_marginal_Q_kernel(env.transition_matrix,env.observation_matrix)




    #for step in range(maximum_depth+1):
    for step in range(maximum_depth):

        if step==0:
            b_reachables[0]=[tuple(initial_mu_state)]

        thisStep_s=Ss[step]
        ls=len(thisStep_s)
        mus=np.reshape(b_reachables[step],(len(b_reachables[step]),len(b_reachables[step][0])))
        #mus=mus[:,1]
        #mus=np.array(list(map(list,mus)))
        z=env.discount_factor**step

        all_current_s=thisStep_s
        all_next_s=Ss[step+1]


        # decompose Mu-beliefs about being in each state (y=0 or 1)
        mus_y0=mus[:,0:ls]
        mus_y1=mus[:,ls:2*ls]
        mus_ys=[mus_y0,mus_y1]

        # Marginal Mu (Mu superscript Y in the paper, or Mu(dy,R)). This variable expresses the probability of being in each state(y)
        marginal_mus_y0=mus_y0.sum(axis=1)
        marginal_mus_y1=mus_y1.sum(axis=1)
        marginal_mus_ys=[marginal_mus_y0,marginal_mus_y1]


        nextStep_uniques=[]
        b_reachables[step+1]=[]
        TK[step]={}


        #for action in range(num_actions):
        for action in range(num_actions):
            for x_prim in range(num_observations):
                ########################################################

                ## prepare

                # C(x,y,a) while C depends only on y
                #rewards/costs of doing action a when the real state is y 
                y0=0
                y1=1
                y_prim0=0
                y_prim1=1
                c_y0=env.rewards[y0][action]
                c_y1=env.rewards[y1][action]
                c=[c_y0,c_y1]

                ### Calculate the denominator of SAY function

                # It is equal to probaility of recieving observation x_prim, regardless of what are the wealth(s) or the state(y). 
                # It calculate sum of probabilities of reaching each state (regardless of wealth level) (marginal_mus_y), while reaciving x_prime observation
                say_denominator=marginal_mus_y0*mq[0][action][x_prim]+marginal_mus_y1*mq[1][action][x_prim]


                ### calculate the nomerator of the SAY function

                # The dimensions of the Mu-space doesn't change with just one action and one observation.  Because in our experiment the reward function is deterministic, all of possible wealths of this step, will transfer to just one other value 
                # based on state and action. So, the size of Mu-space remains constant in SAY calculator function (for doing only one action). Also, the number of possible distributions over Mu, is not a concern for this function: It maps all current possible values to
                # continious values

                ## allocate variables for results of nomerator calculations for each current state

                # these arrays are here to represent Mu distribution of the next state (naturally over its own (next state's) wealth levels) 
                for_y0=np.zeros((len(mus),len(all_next_s)*2))
                for_y1=np.zeros((len(mus),len(all_next_s)*2))
                next_mus=[for_y0,for_y1]

                # tmp_mus[0] for y=0 and tmp_mus[1] for y=1 calculations
                tmp_mus=np.zeros((2,len(mus),len(mus[0])))

                ##
                 # for each current state
                for y,y_mus in enumerate(tmp_mus):

                    # mus_ys is Mu-beliefs about being in a staet (y)
                    # probability of reaching y_prim=0
                    y_mus[:,:len(all_current_s)]=mus_ys[y]*q[y][action][y_prim0][x_prim]
                    # probability of reaching y_prim=1
                    y_mus[:,len(all_current_s):]=mus_ys[y]*q[y][action][y_prim1][x_prim]

                    # Dirac function: d(s+zc(x,y,a))
                    # Here, we compute the possible values of s for next Mu distribution         
                    next_possible_s=np.array(all_current_s)+z*c[y]

                    #The calculated Mu distributions on the S-axis, are defined on the current S values, however these probabilities are for s+zc(y,a).
                    # so as the SAY function inputs are (x,a,x_prim), for each y we can rotate Mu(s) to be matched with next stage's possible values

                    ### indexes of S axis which has value in this SAY calculation
                    ##current_s_indexes=np.where(np.isin(self.S,current_possible_s)==True)[0]

                    # indexes of the next wealth levels which are the successors ( after current S-values recieving c(y,a) )
                    # Here, we used comparison method to check the equality of next time-spte's S-values and current S-values+ z*c 
                    next_s_indexes=np.empty(0,np.int32)
                    for ind,ns in enumerate(next_possible_s):
                        next_related_index=np.where(np.abs(np.array(all_next_s)-ns)<1.0e-5)[0][0]
                        next_s_indexes=np.append(next_s_indexes,next_related_index)


                    #assign Mu-probability to each possible S(=previous_s + c) point
                    # for S points in y_prim=0        
                    next_mus[y][:,next_s_indexes]=y_mus[:,:len(all_current_s)]
                    # for S points in y_prim=1
                    next_mus[y][:,next_s_indexes+len(all_next_s)]=y_mus[:,len(all_current_s):]


                # sum of probabilities of next Mu-space for both conditions : p(mu|y0), p(mu|y1)
                say_nomerator=next_mus[0]+next_mus[1]

                # calculate final SAY result
                # normalize the whole Mu-space dist. by dividing it by totoal probability of taking x_prim
                say_result=say_nomerator/say_denominator[:,None]  

                ### make extended states and transition kernel between them
                for p in range(len(say_result)):


                    ## reachable extended states
                    redundants=[]
                    is_unique=True
                    for i in range(len(nextStep_uniques)):
                        if np.sum(np.abs(np.array(say_result[p])-np.array(nextStep_uniques[i])))<comparison_precision:
                            is_unique=False
                            redundants.append(i)
                    
                    if is_unique:

                        # add unique values for other comparisons
                        #nextStep_uniques[0].extend([x_prim])
                        nextStep_uniques.append(say_result[p])

                        # adding to extended reachable states
                        b_reachables[step+1].append((tuple(say_result[p])))

                        ######### transition kernel of extended states

                        ex_state=tuple(say_result[p])
                        # here x_prime means x. (should be checked)
                        ex_kernel_element=(x_prim,b_reachables[step][p],action)
                        
                        TK[step][ex_kernel_element]=ex_state
                    else:
                        ## transition kernel of extended states

                        ex_state=tuple(nextStep_uniques[redundants[0]])
                        # here x_prime means x. (should be checked)
                        ex_kernel_element=(x_prim,b_reachables[step][p],action)
                        
                        TK[step][ex_kernel_element]=ex_state
                        
    #################################################################################### VI
    value_func={}
    action_func={}
    full_val_func={}

    for step in range(maximum_depth,-1,-1):
        points=b_reachables[step]
        #value_func[step]={}
        #full_val_func[step]={}
        if step==maximum_depth:
            each_wealth_level_value=b_utility(Ss[step],exp_weights,exp_vorfaktoren)

            each_mu_value=np.dot(np.array(list(map(list,points))),np.hstack([each_wealth_level_value,each_wealth_level_value]).T)

            value_func[step]=np.hstack([dict(zip(points,each_mu_value.reshape(-1))).copy(),dict(zip(points,each_mu_value.reshape(-1))).copy()])

            full_val_func[step]=[dict(zip(points,each_mu_value.reshape(-1))).copy()]
        else:
            val_x0=np.zeros((len(points),num_actions))
            val_x1=np.zeros((len(points),num_actions))
            val_x=[val_x0,val_x1]
            for x in observations:
                for a in range(num_actions):
                    for p,point in enumerate(points):

                        val_x[x][p][a]=value_func[step+1][x][TK[step][(x,point,a)]]

                to_v0=np.max(val_x0,axis=1)
                to_a0=np.argmax(val_x0,axis=1)

                to_v1=np.max(val_x1,axis=1)
                to_a1=np.argmax(val_x1,axis=1)

            value_func[step]=np.hstack([dict(zip(points,to_v0)).copy(),dict(zip(points,to_v1)).copy()])
            action_func[step]=np.hstack([dict(zip(points,to_a0)).copy(),dict(zip(points,to_a1)).copy()])
            full_val_func[step]=[dict(zip(points,val_x0)).copy()]
    
    return Ss,b_reachables,TK,full_val_func,value_func,action_func



## Initialize for simulation

In [6]:
e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)
e.reset()
initial_observation=[e.current_state]
observation=initial_observation
state=e.current_state
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 0.9, 'initial_wealth': 0}


In [6]:
max_depth=2
exp_vorfaktoren=[0.2,0.3]
exp_weights=[2,1]

In [7]:
m_r,m_map, m_q_func,m_value_function,m_action_func=MO_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_theta=[0.5,0.5],initial_observation=0,rounding_prec_coeff=10000)

b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=Beurele_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_mu_state=[0.5,0.5],initial_wealth=0,comparison_precision=1.0e-5)

In [ ]:
E=[-1.5,-0.2,-0.1,0.1,0.2,1.5]
W=[-1,0,1,2]
inits=[[1,0],[0.9,0.1],[0.75,0.25],[0.5,0.5]]

loop_counts=0
for max_depth in range(1,5):
    corr_count=0
    all_count=0
    loop_counts+=1
    print(loop_counts)
    for e1 in E:
        for e2 in E:
            exp_vorfaktoren=[e1,e2]
            for w1 in W:
                for w2 in W:
                    exp_weights=[w1,w2]
                    for init in inits:
                        m_r,m_map, m_q_func,m_value_function,m_action_func=MO_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_theta=init,initial_observation=0,rounding_prec_coeff=10000)
                        b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=Beurele_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_mu_state=init,initial_wealth=0,comparison_precision=1.0e-5)
                        
                        m_init=init*len(init)
                        m_init.append(0)
                        
                        all_count+=1
                        if m_action_func[0][tuple(m_init)]==b_action_func[0][0][tuple(init)]:
                            corr_count+=1
                        else:
                            print (m_q_func[0][tuple(m_init)], b_q_func[0][0][tuple(init)],'           ',m_action_func[0][tuple(m_init)],b_action_func[0][0][tuple(init)],'    ||     exp:',exp_vorfaktoren,exp_weights,init)
    print('--------------------------')                        
    print(all_count-corr_count,corr_count,corr_count/all_count)
    print('=====================================')

1
--------------------------
0 2304 1.0
2
[ -6.26295986 -11.92226639  -5.10857213 -39.97452391  -7.91732414] [ -2.40761482 -15.93158022  -2.54926001 -71.02713903  -8.06574157]             2 0     ||     exp: [-1.5, -1.5] [-1, -1] [0.9, 0.1]
[ -6.93435882 -10.64469872  -6.38613981 -33.96379895 -13.92804909] [ -3.46163101 -13.42239518  -5.05844505 -59.22187701 -19.87100359]             2 0     ||     exp: [-1.5, -1.5] [-1, -1] [0.75, 0.25]
[ -3.13147993  -5.96113319  -2.55428607 -19.98726195  -3.95866207] [ -1.20380741  -7.96579011  -1.27463001 -35.51356952  -4.03287079]             2 0     ||     exp: [-1.5, -1.5] [-1, 0] [0.9, 0.1]
[ -3.46717941  -5.32234936  -3.1930699  -16.98189947  -6.96402455] [ -1.73081551  -6.71119759  -2.52922252 -29.6109385   -9.9355018 ]             2 0     ||     exp: [-1.5, -1.5] [-1, 0] [0.75, 0.25]
[ -3.13147993  -5.96113319  -2.55428607 -19.98726195  -3.95866207] [ -1.20380741  -7.96579011  -1.27463001 -35.51356952  -4.03287079]             2 0     ||    

In [84]:
exp_vorfaktoren

[-1.5, -1.5]

In [12]:
init

[0.9, 0.1]

In [14]:
b_q_func[0][0]

{(0.9,
  0.1): array([ -2.40761482, -15.93158022,  -2.54926001, -71.02713903,
         -8.06574157])}

In [70]:
def both_actions(m_current_theta,b_current_mu,b_current_x,time_step):
    return m_action_func[time_step][m_current_theta],b_current_state[time_step][b_current_x][b_current_mu]

In [ ]:
ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='naive')
ag.reset()
x_map,M,F,G,X,value_function,all_theta=ag.pre_planning(exp_vorfaktoren, initial_theta=initial_theta,initial_observation=initial_observation)
ag.value_iteration()
for t in range(maximum_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')
    

In [69]:
b_action_func

{1: array([{(0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.30000000000000004, 0.0, 0.0): 4, (0.0, 0.0, 0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0): 3, (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0): 3, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25): 3},
        {(0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.30000000000000004, 0.0, 0.0): 4, (0.0, 0.0, 0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0): 3, (0.0, 0.25, 0.0, 0.25, 0.0, 0.0, 0.25, 0.0, 0.25, 0.0): 3, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25): 3}],
       dtype=object),
 0: array([{(0.5, 0.5): 3}, {(0.5, 0.5): 3}], dtype=object)}

In [ ]:
E=[-1.5,-0.2,-0.1,0.1,0.2,1.5]
W=[-1,0,1,2]
inits=[[1,0],[0.9,0.1],[0.75,0.25],[0.5,0.5]]

loop_counts=0
for max_depth in range(1,5):
    corr_count=0
    all_count=0
    loop_counts+=1
    print(loop_counts)
    for e1 in E:
        for e2 in E:
            exp_vorfaktoren=[e1,e2]
            for w1 in W:
                for w2 in W:
                    exp_weights=[w1,w2]
                    for init in inits:
                        m_r,m_map, m_q_func,m_value_function,m_action_func=MO_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_theta=init,initial_observation=0,rounding_prec_coeff=10000)
                        b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=Beurele_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_mu_state=init,initial_wealth=0,comparison_precision=1.0e-5)
                        
                        m_init=init*len(init)
                        m_init.append(0)
                        
                        all_count+=1
                        if m_action_func[0][tuple(m_init)]==b_action_func[0][0][tuple(init)]:
                            corr_count+=1
                        else:
                            print (m_q_func[0][tuple(m_init)], b_q_func[0][0][tuple(init)],'           ',m_action_func[0][tuple(m_init)],b_action_func[0][0][tuple(init)],'    ||     exp:',exp_vorfaktoren,exp_weights,init)
    print('--------------------------')                        
    print(all_count-corr_count,all_count,corr_count/all_count)
    print('=====================================')

1
--------------------------
0 2304 1.0
2
[ -6.26295986 -11.92226639  -5.10857213 -39.97452391  -7.91732414] [ -2.40761482 -15.93158022  -2.54926001 -71.02713903  -8.06574157]             2 0     ||     exp: [-1.5, -1.5] [-1, -1] [0.9, 0.1]
[ -6.93435882 -10.64469872  -6.38613981 -33.96379895 -13.92804909] [ -3.46163101 -13.42239518  -5.05844505 -59.22187701 -19.87100359]             2 0     ||     exp: [-1.5, -1.5] [-1, -1] [0.75, 0.25]
[ -3.13147993  -5.96113319  -2.55428607 -19.98726195  -3.95866207] [ -1.20380741  -7.96579011  -1.27463001 -35.51356952  -4.03287079]             2 0     ||     exp: [-1.5, -1.5] [-1, 0] [0.9, 0.1]
[ -3.46717941  -5.32234936  -3.1930699  -16.98189947  -6.96402455] [ -1.73081551  -6.71119759  -2.52922252 -29.6109385   -9.9355018 ]             2 0     ||     exp: [-1.5, -1.5] [-1, 0] [0.75, 0.25]
[ -3.13147993  -5.96113319  -2.55428607 -19.98726195  -3.95866207] [ -1.20380741  -7.96579011  -1.27463001 -35.51356952  -4.03287079]             2 0     ||    

In [10]:
a=np.array([[1,2,3,4],[4,3,5,2]])


In [3]:
a.shape

(2, 4)

In [11]:
b=np.sum(a,axis=0).reshape(1,4)

In [14]:
a/b

array([[0.2       , 0.4       , 0.375     , 0.66666667],
       [0.8       , 0.6       , 0.625     , 0.33333333]])

# MO

In [16]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
from MO_agent import *
import random
import agent as agent
from MO_agent import *

e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 0.9, 'initial_wealth': 0}


In [17]:
e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))

In [29]:
planning_depth=2
partitioning_chunk_number=1000
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
partitioning_chunk_number=1000
initial_theta=[0.5,0.5]

In [26]:
ag3=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='discrete_optimized')
ag3.reset()
x_map3,M3,F3,G3,X3,value_function3,all_theta3=ag3.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation)
v3,a3,q3,vf3=ag3.value_iteration()

this function has not been finilized!


In [13]:

ag1=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='naive')
ag1.reset()
x_map1,M1,F1,G1,X1,value_function1,all_theta1=ag1.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation)
v1,a1,q1,vf1=ag1.value_iteration()

In [30]:
ag2=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='cheating')
ag2.reset()
x_map2,M2,F2,G2,X2,value_function2,all_theta2=ag2.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation)
v2,a2,q2,vf2=ag2.value_iteration()

In [8]:
ag=ag2
for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: [0.5, 0.5, 0.5, 0.5, 0]

***** [0.3, 0.7, 0.3, 0.7, 1]
             action: listen
             **
reward: -0.5 new observation: 1

new internal state: [0.3, 0.7, 0.3, 0.7, 1]

t= 1
------
state: 0 tiger_right
last_observation: 1
-------------------------------

current internal state: [0.3, 0.7, 0.3, 0.7, 1]

***** [0.5, 0.5, 0.5, 0.5, 1]
             action: open_right_low
             **
reward: -1.0 new observation: 1

new internal state: [0.5, 0.5, 0.5, 0.5, 1]



# B

In [1]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
import random
from Beurele_agent import *
from MO_agent import *


e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 0.9, 'initial_wealth': 0}


In [2]:
e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)
e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 0.9, 'initial_wealth': 0}


In [3]:
max_depth=2
planning_depth=max_depth
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
initial_Mu=[0.5,0.5]
initial_observation=0

In [5]:
ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)

In [6]:
b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)

In [4]:
ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)
b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)

ag.ch_reset(initial_Mu,0,initial_observation)
for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_belief)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag.ch_do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.ch_update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_belief)
    print('')
    print('============================')

t= 0
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: [0.5, 0.5]

             action: listen
             **
reward: -0.5 new observation: 1

new internal state: [0.0, 0.0, 0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0]

t= 1
------
state: 0 tiger_right
last_observation: 1
-------------------------------

current internal state: [0.0, 0.0, 0.30000000000000004, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0]

             action: open_right_low
             **
reward: -1.0 new observation: 0

new internal state: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]



1
--------------------------
0 2304 1.0
2
[ -6.26295986 -11.92226639  -5.10857213 -39.97452391  -7.91732414] [ -2.40761482 -15.93158022  -2.54926001 -71.02713903  -8.06574157]             2 0     ||     exp: [-1.5, -1.5] [-1, -1] [0.9, 0.1]
[ -6.93435882 -10.64469872  -6.38613981 -33.96379895 -13.92804909] [ -3.46163101 -13.42239518  -5.05844505 -59.22187701 -19.87100359]             2 0     ||     exp: [-1.5, -1.5] [-1, -1] [0.75, 0.25]
[ -3.13147993  -5.96113319  -2.55428607 -19.98726195  -3.95866207] [ -1.20380741  -7.96579011  -1.27463001 -35.51356952  -4.03287079]             2 0     ||     exp: [-1.5, -1.5] [-1, 0] [0.9, 0.1]
[ -3.46717941  -5.32234936  -3.1930699  -16.98189947  -6.96402455] [ -1.73081551  -6.71119759  -2.52922252 -29.6109385   -9.9355018 ]             2 0     ||     exp: [-1.5, -1.5] [-1, 0] [0.75, 0.25]
[ -3.13147993  -5.96113319  -2.55428607 -19.98726195  -3.95866207] [ -1.20380741  -7.96579011  -1.27463001 -35.51356952  -4.03287079]             2 0     ||    